# Capstone Project - The Battle of Neighborhoods (Week 1)

Writer: Carlo K.

Date: 15.03.2021

## 1. Introduction

Bangkok, situated as a metropolitan city in the center of Thailand and South-East Asian region, is a very multicultural and diverse city. As Bangkok becomes a popular tourist destination, it offers wide range of cultural experiences and opportunities for a lot of business to run.

## 2. Business Problem

The aim is to identify where an investor should open a restaurant or a café, depending on their target group of customers such as - in the central area where business districts are located? or in the popular area for the tourist?

This helps the investors to make decision to pick the location suitable most for their business. Our findings will also include the different kinds of cuisines, hotels, business districts that they should be focused on.

## 3. Data

### 3.1 Districts in Bangkok

The districts in Bangkok data is available on: [https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok](https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok)

The data that is provided:
- *District (Khet)* : Name of Neighborhood
- *Map Nr*
- *Postcode*
- *Name in Thai scripts*
- *Population*
- *No. of Subdistricts (Khwaeng)*
- *Latitude* : Latitude of the Neighbourhood
- *Longitude* : Longitude of the Neighbourhood

The data is complete with latitude and longtitude of each district, therefore we can scrape all the data we need directly from the website.

### 3.2 Data from Foursquare API

Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

We connect to the Foursquare API to gather information about venues inside each and every neighbourhood. For each neighbourhood, we have chosen the radius to be 500 meters.

The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

- Neighbourhood : Name of the Neighbourhood
- Neighbourhood Latitude : Latitude of the Neighbourhood
- Neighbourhood Longitude : Longitude of the Neighbourhood
- Venue : Name of the Venue
- Venue Latitude : Latitude of Venue
- Venue Longitude : Longitude of Venue
- Venue Category : Category of Venue

Based on all the information collected for Bangkok, we have sufficient data to build our model. We cluster the neighbourhoods together based on similar venue categories. We then present our observations and findings. Using this data, our stakeholders can take the necessary decision.

## 4. Methodology

We import all necessary packages to support the analysis:

- *requests* : Handle http requests
- *pandas* : Manipulate data and data analysis
- *folium* : Generate maps
- *matplotlib* : Manipulate details of maps
- *sklearn* : KMeans to clustering the data

### 4.1 Collecting data

In [10]:
import requests
import pandas as pd
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

Collecting data of Bangkok

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok'
page = requests.get(url)

Clean the data andmake it ready for further analysis

In [8]:
table = pd.read_html(page.text)
table = table[0]
table.drop(['MapNr', 'Thai', 'Popu-lation', 'No. ofSubdis-trictsKhwaeng'], axis=1, inplace=True)
table.columns = ['Neighborhood', 'PostalCode', 'Latitude', 'Longitude']
table

,Neighborhood,PostalCode,Latitude,Longitude
0,Bang Bon,10150,13.659200,100.399100
1,Bang Kapi,10240,13.765833,100.647778
2,Bang Khae,10160,13.696111,100.409444
3,Bang Khen,10220,13.873889,100.596389
4,Bang Kho Laem,10120,13.693333,100.502500
5,Bang Khun Thian,10150,13.660833,100.435833
6,Bang Na,10260,13.680081,100.591800
7,Bang Phlat,10700,13.793889,100.505000
8,Bang Rak,10500,13.730833,100.524167
9,Bang Sue,10800,13.809722,100.537222


In [11]:
address = 'Bangkok'

geolocator = Nominatim(user_agent="bangkok_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangkok are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangkok are 13.7544238, 100.4930399.


In [15]:
# create map of Bangkok using latitude and longitude values
map_bangkok = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(table['Latitude'], table['Longitude'], table['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangkok)  
    
map_bangkok

In [16]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'XVJ1G2ITSHC3DPF5YA0FJZ0TCMBELT040TUVKPYAHJN0XTDV' # your Foursquare ID
CLIENT_SECRET = 'CQFQBN0NAFQJYFA0M5CADUWHNT3ACRMWWB0OMMB4Q2BONCSU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XVJ1G2ITSHC3DPF5YA0FJZ0TCMBELT040TUVKPYAHJN0XTDV
CLIENT_SECRET:CQFQBN0NAFQJYFA0M5CADUWHNT3ACRMWWB0OMMB4Q2BONCSU


In [17]:
# (From Lab) Create a function to repeat the same process to all the neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
bangkok_venues = getNearbyVenues(table['Neighborhood'], table['Latitude'], table['Longitude'], radius=500)

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [20]:
# Check size
print(bangkok_venues.shape)
bangkok_venues.head()

(1148, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Thai Restaurant
1,Bang Bon,13.6592,100.3991,Irashaimase Japanese Restaurant,13.658358,100.401403,Japanese Restaurant
2,Bang Bon,13.6592,100.3991,Lotus Express (โลตัส เอ็กซ์เพรส),13.657839,100.397243,Convenience Store
3,Bang Bon,13.6592,100.3991,ส.ทิพรส,13.659368,100.399382,Noodle House
4,Bang Bon,13.6592,100.3991,บ้านพลูหลวง เอกชัย,13.658482,100.398440,Asian Restaurant


In [21]:
# Check how many venues were returned for each neighborhood
bangkok_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bang Bon,8,8,8,8,8,8
Bang Kapi,21,21,21,21,21,21
Bang Khae,11,11,11,11,11,11
Bang Khen,15,15,15,15,15,15
Bang Kho Laem,16,16,16,16,16,16
Bang Khun Thian,39,39,39,39,39,39
Bang Na,9,9,9,9,9,9
Bang Phlat,11,11,11,11,11,11
Bang Rak,28,28,28,28,28,28


In [22]:
# one hot encoding
bangkok_onehot = pd.get_dummies(bangkok_venues[['Venue Category']], prefix="", prefix_sep="")
bangkok_onehot.shape

(1148, 162)

In [23]:
# add neighborhood column back to dataframe
bangkok_onehot['Neighborhood'] = bangkok_venues['Neighborhood'] 

bangkok_onehot.columns.get_loc("Neighborhood")

104

In [24]:
# The column didn't go to the back
# Manually move to the first column
fixed_columns = [bangkok_onehot.columns[104]] + list(bangkok_onehot.columns[:104]) + list(bangkok_onehot.columns[105:])
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,...,Theater,Tonkatsu Restaurant,Tour Provider,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bang Bon,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
bangkok_grouped = bangkok_onehot.groupby('Neighborhood').mean().reset_index()
bangkok_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,...,Theater,Tonkatsu Restaurant,Tour Provider,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bang Bon,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bang Kapi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.047619,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bang Khae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000
3,Bang Khen,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000
4,Bang Kho Laem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000
5,Bang Khun Thian,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.025641,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bang Na,0.111111,0.000000,0.000000,0.000000,0.222222,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Bang Phlat,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Bang Rak,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bang Sue,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in bangkok_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bangkok_grouped[bangkok_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bang Bon----
                 venue  freq
0         Noodle House  0.12
1  Japanese Restaurant  0.12
2     Asian Restaurant  0.12
3      Automotive Shop  0.12
4    Convenience Store  0.12


----Bang Kapi----
               venue  freq
0       Noodle House  0.19
1  Convenience Store  0.10
2          Multiplex  0.05
3    Thai Restaurant  0.05
4        Coffee Shop  0.05


----Bang Khae----
                 venue  freq
0    Convenience Store  0.27
1         Noodle House  0.18
2          Wings Joint  0.09
3  Japanese Restaurant  0.09
4          Coffee Shop  0.09


----Bang Khen----
                   venue  freq
0       Asian Restaurant  0.20
1           Noodle House  0.13
2  Vietnamese Restaurant  0.13
3      Convenience Store  0.13
4                 Garden  0.07


----Bang Kho Laem----
                venue  freq
0        Noodle House  0.25
1     Thai Restaurant  0.12
2  Chinese Restaurant  0.12
3         Coffee Shop  0.12
4   Hotpot Restaurant  0.06


----Bang Khun Thian----
         

In [27]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangkok_grouped['Neighborhood']

for ind in np.arange(bangkok_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangkok_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Convenience Store,Shopping Plaza,Asian Restaurant,Automotive Shop,Grocery Store,Thai Restaurant,Japanese Restaurant,Noodle House,Electronics Store,Food
1,Bang Kapi,Noodle House,Convenience Store,Market,Gay Bar,Massage Studio,Som Tum Restaurant,Shabu-Shabu Restaurant,Multiplex,Museum,Steakhouse
2,Bang Khae,Convenience Store,Noodle House,Wings Joint,Coffee Shop,Shopping Mall,Fast Food Restaurant,Japanese Restaurant,Supermarket,Electronics Store,Food
3,Bang Khen,Asian Restaurant,Convenience Store,Vietnamese Restaurant,Noodle House,Other Nightlife,Som Tum Restaurant,Bus Stop,Garden,Garden Center,Café
4,Bang Kho Laem,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Supermarket,Convenience Store,Hotpot Restaurant,Vietnamese Restaurant


In [29]:
# set number of clusters
kclusters = 5

bangkok_grouped_clustering = bangkok_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangkok_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

In [30]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 1, 3, 0, 3, 1], dtype=int32)

In [31]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangkok_merged = table

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
bangkok_merged = bangkok_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bangkok_merged.head() # check the last columns!

,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,10150,13.659200,100.399100,0,Convenience Store,Shopping Plaza,Asian Restaurant,Automotive Shop,Grocery Store,Thai Restaurant,Japanese Restaurant,Noodle House,Electronics Store,Food
1,Bang Kapi,10240,13.765833,100.647778,0,Noodle House,Convenience Store,Market,Gay Bar,Massage Studio,Som Tum Restaurant,Shabu-Shabu Restaurant,Multiplex,Museum,Steakhouse
2,Bang Khae,10160,13.696111,100.409444,0,Convenience Store,Noodle House,Wings Joint,Coffee Shop,Shopping Mall,Fast Food Restaurant,Japanese Restaurant,Supermarket,Electronics Store,Food
3,Bang Khen,10220,13.873889,100.596389,0,Asian Restaurant,Convenience Store,Vietnamese Restaurant,Noodle House,Other Nightlife,Som Tum Restaurant,Bus Stop,Garden,Garden Center,Café
4,Bang Kho Laem,10120,13.693333,100.502500,3,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Supermarket,Convenience Store,Hotpot Restaurant,Vietnamese Restaurant


In [32]:
bangkok_merged_nanrm = bangkok_merged.dropna(subset=['Cluster Labels'])

In [33]:
# Visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangkok_merged_nanrm['Latitude'], bangkok_merged_nanrm['Longitude'], bangkok_merged_nanrm['Neighborhood'], bangkok_merged_nanrm['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

### Cluster 1

In [34]:
bangkok_merged_nanrm.loc[bangkok_merged_nanrm['Cluster Labels'] == 0, bangkok_merged_nanrm.columns[[1] + list(range(5, bangkok_merged_nanrm.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10150,Convenience Store,Shopping Plaza,Asian Restaurant,Automotive Shop,Grocery Store,Thai Restaurant,Japanese Restaurant,Noodle House,Electronics Store,Food
1,10240,Noodle House,Convenience Store,Market,Gay Bar,Massage Studio,Som Tum Restaurant,Shabu-Shabu Restaurant,Multiplex,Museum,Steakhouse
2,10160,Convenience Store,Noodle House,Wings Joint,Coffee Shop,Shopping Mall,Fast Food Restaurant,Japanese Restaurant,Supermarket,Electronics Store,Food
3,10220,Asian Restaurant,Convenience Store,Vietnamese Restaurant,Noodle House,Other Nightlife,Som Tum Restaurant,Bus Stop,Garden,Garden Center,Café
7,10700,Bar,Convenience Store,Music Venue,Intersection,Cocktail Bar,Clothing Store,Bus Stop,Fast Food Restaurant,Café,Farmers Market
12,10240,Park,Convenience Store,Miscellaneous Shop,Gourmet Shop,Market,Shop & Service,Duty-free Shop,Floating Market,Flea Market,Fast Food Restaurant
15,10400,Convenience Store,Recreation Center,Stadium,Department Store,Park,Bus Stop,Skate Park,Fried Chicken Joint,Sports Club,Food Court
16,10210,Convenience Store,Spa,Hotel,Gym,Buffet,Thai Restaurant,Restaurant,Food Truck,Auto Garage,Diner
27,10530,Park,Other Repair Shop,Convenience Store,Food,Flea Market,Shopping Mall,Floating Market,Fast Food Restaurant,Farmers Market,Electronics Store
29,10330,Convenience Store,Noodle House,Asian Restaurant,Seafood Restaurant,Dessert Shop,Chinese Restaurant,Thai Restaurant,Japanese Restaurant,Hotpot Restaurant,Ice Cream Shop


### Cluster 2

In [35]:
bangkok_merged_nanrm.loc[bangkok_merged_nanrm['Cluster Labels'] == 1, bangkok_merged_nanrm.columns[[1] + list(range(5, bangkok_merged_nanrm.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,10150,Thai Restaurant,Japanese Restaurant,Dessert Shop,Clothing Store,Bakery,Pizza Place,Restaurant,Noodle House,Gym / Fitness Center,Café
9,10800,Thai Restaurant,Coffee Shop,Noodle House,Convenience Store,Badminton Court,Seafood Restaurant,Bar,Playground,Hotpot Restaurant,Gourmet Shop
13,10900,Coffee Shop,Thai Restaurant,Flea Market,Convenience Store,Bubble Tea Shop,Massage Studio,Seafood Restaurant,Café,Fast Food Restaurant,Noodle House
19,10230,Japanese Restaurant,Bakery,Coffee Shop,Ice Cream Shop,Shabu-Shabu Restaurant,Steakhouse,Fast Food Restaurant,Multiplex,BBQ Joint,Shopping Mall
20,10510,Pub,Coffee Shop,Buffet,Music Venue,Thai Restaurant,Japanese Restaurant,Chinese Restaurant,Convenience Store,Diner,Dim Sum Restaurant
21,10600,Coffee Shop,Thai Restaurant,Dessert Shop,Art Gallery,Café,Noodle House,Hotel Bar,Historic Site,Hotel,Clothing Store
22,10110,Bar,Recording Studio,Thai Restaurant,Bubble Tea Shop,Karaoke Bar,Canal,Discount Store,Farmers Market,Department Store,Food & Drink Shop
23,10210,Coffee Shop,Steakhouse,Noodle House,Fast Food Restaurant,Thai Restaurant,Japanese Restaurant,Convenience Store,Canal,Café,Bus Stop
25,10230,Noodle House,Asian Restaurant,Café,Coffee Shop,Som Tum Restaurant,Hotel,Massage Studio,Hotpot Restaurant,BBQ Joint,Badminton Court
26,10510,Thai Restaurant,Department Store,Coffee Shop,Intersection,Duty-free Shop,Food & Drink Shop,Food,Floating Market,Flea Market,Fast Food Restaurant


### Cluster 3

In [36]:
bangkok_merged_nanrm.loc[bangkok_merged_nanrm['Cluster Labels'] == 2, bangkok_merged_nanrm.columns[[1] + list(range(5, bangkok_merged_nanrm.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,10150,Convenience Store,Coffee Shop,Diner,Thai Restaurant,Dessert Shop,Farmers Market,Food Court,Food & Drink Shop,Food,Floating Market
35,10250,Convenience Store,Comfort Food Restaurant,Halal Restaurant,Noodle House,Women's Store,Electronics Store,Food & Drink Shop,Food,Floating Market,Flea Market
40,10240,Convenience Store,Stadium,Japanese Restaurant,Thai Restaurant,Duty-free Shop,Food & Drink Shop,Food,Floating Market,Flea Market,Fast Food Restaurant


### Cluster 4

In [37]:
bangkok_merged_nanrm.loc[bangkok_merged_nanrm['Cluster Labels'] == 3, bangkok_merged_nanrm.columns[[1] + list(range(5, bangkok_merged_nanrm.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,10120,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Supermarket,Convenience Store,Hotpot Restaurant,Vietnamese Restaurant
6,10260,Asian Restaurant,Noodle House,American Restaurant,Seafood Restaurant,Intersection,Chinese Restaurant,Café,Food,Floating Market,Flea Market
8,10500,Noodle House,Hotel,Chinese Restaurant,Thai Restaurant,Hotel Bar,Soup Place,Massage Studio,Café,Seafood Restaurant,Shabu-Shabu Restaurant
10,10700,Noodle House,Som Tum Restaurant,Convenience Store,Steakhouse,Thai Restaurant,Dim Sum Restaurant,Coffee Shop,Park,Café,Dessert Shop
11,10600,Noodle House,Coffee Shop,Dessert Shop,Asian Restaurant,Farmers Market,Seafood Restaurant,Photography Studio,Train Station,Soup Place,BBQ Joint
17,10300,Noodle House,Convenience Store,Asian Restaurant,Som Tum Restaurant,Dessert Shop,Coffee Shop,Market,Tennis Court,Discount Store,Donburi Restaurant
18,10310,Noodle House,Hotel,Asian Restaurant,Som Tum Restaurant,Bed & Breakfast,Restaurant,Supermarket,Bistro,BBQ Joint,Convenience Store
41,10120,Noodle House,Asian Restaurant,Dessert Shop,Chinese Restaurant,Convenience Store,Coffee Shop,Café,Thai Restaurant,Bakery,Food Truck
42,10250,Noodle House,Convenience Store,Asian Restaurant,Café,Coffee Shop,German Restaurant,Farmers Market,Som Tum Restaurant,Massage Studio,Chinese Restaurant
43,10170,Noodle House,Convenience Store,Seafood Restaurant,Soccer Field,Floating Market,Asian Restaurant,Food,Flea Market,Fast Food Restaurant,Farmers Market


### Cluster 5

In [38]:
bangkok_merged_nanrm.loc[bangkok_merged_nanrm['Cluster Labels'] == 4, bangkok_merged_nanrm.columns[[1] + list(range(5, bangkok_merged_nanrm.shape[1]))]]

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,10520,Thai Restaurant,Noodle House,Café,Som Tum Restaurant,Hotel,Asian Restaurant,Steakhouse,Electronics Store,Food & Drink Shop,Food
36,10140,Thai Restaurant,Noodle House,Som Tum Restaurant,Hotpot Restaurant,Asian Restaurant,Bistro,Women's Store,Farmers Market,Food & Drink Shop,Food
38,10220,Thai Restaurant,Bar,Noodle House,Grocery Store,Women's Store,Food & Drink Shop,Food,Floating Market,Flea Market,Fast Food Restaurant
